In [21]:
import tensorflow.keras as keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold

import time
from statistics import mean


def load_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0
    y_train_onehot = keras.utils.to_categorical(y_train)
    y_test_onehot = keras.utils.to_categorical(y_test)
    return X_train, y_train_onehot, X_test, y_test_onehot


class Model:

    def __init__(self):
        self.model = Sequential()

    def build(self):
        self.model.add(Conv2D(32, 3, activation='relu', kernel_initializer=GlorotNormal(), input_shape=(28, 28, 1)))
        self.model.add(MaxPooling2D())
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(10, activation='softmax'))
        self.model.compile(optimizer=SGD(momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

    def validate(self, X, Y, K):
        train_accs = []
        test_accs = []
        kfold = KFold(K, shuffle=True, random_state=1)
        k_count = 1
        for i_train, i_test in kfold.split(X):
            X_train, y_train, X_test, y_test = X[i_train], Y[i_train], X[i_test], Y[i_test]
            self.model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

            _, train_acc = self.model.evaluate(X_train, y_train)
            train_accs.append(train_acc)

            _, test_acc = self.model.evaluate(X_test, y_test)
            test_accs.append(test_acc)

            k_count += 1

        return train_accs, test_accs


start = time.time()
X_train, y_train, X_test, y_test = load_fashion_mnist()
model = Model()
model.build()
train_accs, test_accs = model.validate(X_train, y_train, 5)
print(f'Train Accuracy: {round(mean(train_accs) * 100, 2)}%')
print(f'Test Accuracy: {round(mean(test_accs) * 100, 2)}%')
runtime = round(int(time.time() - start) / 60, 2)
print(f'--- {runtime}m ---')


Epoch 1/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.6003 - accuracy: 0.7773 - val_loss: 0.3970 - val_accuracy: 0.8543
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3647 - accuracy: 0.8668 - val_loss: 0.3543 - val_accuracy: 0.8723
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3148 - accuracy: 0.8835 - val_loss: 0.3132 - val_accuracy: 0.8850
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2865 - accuracy: 0.8929 - val_loss: 0.3103 - val_accuracy: 0.8831
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2635 - accuracy: 0.9015 - val_loss: 0.2743 - val_accuracy: 0.9005
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2453 - accuracy: 0.9086 - val_loss: 0.2744 - val_accuracy: 0.8980
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2303 - accuracy: 0.9129 - val_loss: 0.2612 - val_accuracy: